# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession
 # 5) write code to answer the quiz questions 

In [2]:
# 1) import any other libraries you might need
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# 2) instantiate a Spark session 
spark = SparkSession \
    .builder \
    .appName("Data wrangling with Spark SQL") \
    .getOrCreate()

In [4]:
# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [5]:
# 4) create a view to use with your SQL queries
user_log.createOrReplaceTempView("user_log_table")

In [6]:
spark.sql("SELECT * FROM user_log_table LIMIT 2").show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

# Question 1

Which page did user id ""(empty string) NOT visit?

In [7]:
# write your code to answer question 1
# Right join the results to find pages that blank visitor did not visit
spark.sql(
        '''
            SELECT *
            FROM (
            SELECT DISTINCT page
                FROM user_log_table
                WHERE userID='') AS user_pages
            RIGHT JOIN (
                SELECT DISTINCT page
                FROM user_log_table) AS all_pages
            ON user_pages.page = all_pages.page
            WHERE user_pages.page IS NULL
        ''').show()

+----+----------------+
|page|            page|
+----+----------------+
|null|Submit Downgrade|
|null|       Downgrade|
|null|          Logout|
|null|   Save Settings|
|null|        Settings|
|null|        NextSong|
|null|         Upgrade|
|null|           Error|
|null|  Submit Upgrade|
+----+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?
Depending on the knowledge

# Question 3

How many female users do we have in the data set?

In [8]:
# write your code to answer question 3
spark.sql('''
            SELECT COUNT(DISTINCT userID)
            FROM user_log_table
            WHERE gender = 'F'
          ''').show()

+----------------------+
|count(DISTINCT userID)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [9]:
# write your code to answer question 4

spark.sql('''
            SELECT Artist, COUNT(Artist) AS plays
            FROM user_log_table
            GROUP BY Artist
            ORDER BY plays DESC
            LIMIT 1
          ''').show()

+--------+-----+
|  Artist|plays|
+--------+-----+
|Coldplay|   83|
+--------+-----+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [10]:
# TODO: write your code to answer question 5

# SELECT CASE WHEN 1 > 0 THEN 1 WHEN 2 > 0 THEN 2.0 ELSE 1.2 END;
is_home = spark.sql('''
                    SELECT userID, page, ts, 
                    CASE WHEN page = 'Home' THEN 1 ELSE 0 
                    END AS is_home FROM user_log_table
                    WHERE (page = 'NextSong') or (page = 'Home')
                    ''')

# keep the results in a new view
is_home.createOrReplaceTempView("is_home_table")

# find the cumulative sum over the is_home column
cumulative_sum = spark.sql('''
                            SELECT *, SUM(is_home) 
                            OVER (PARTITION BY userID ORDER BY ts DESC
                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period
                            FROM is_home_table''')

# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

# find the average count for NextSong
spark.sql('''
            SELECT AVG(count_results) 
            FROM (
                SELECT COUNT(*) AS count_results 
                FROM period_table
                GROUP BY userID, period, page 
                HAVING page = 'NextSong'
                ) AS counts
            ''').show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

